# Question_1

### import lib

In [89]:
from nltk.stem import WordNetLemmatizer
import numpy as np
import random
from sklearn.linear_model import LogisticRegression as LGR
from sklearn.cluster import KMeans

### PreProcess data

In [90]:
"""
train_data includes 2400 samples, where each sample is a list including the
elements which are the words in reviews.

train_label includes 2400 samples which belongs to {0,1}, which is the label 
of train_data.

test_data has the same form as the train_data, while it has 600 sample.

test_label is the same as train_label.
"""
def Split(filenames):

    train_data = []
    train_label = []
    test_data = []
    test_label = []
    root = "sentiment labelled sentences/"
    for filename in filenames:
        path = root + filename
        count = [1,1]
        punctuation = ["!","%","&","(",")","+",".",":",";","<","=",">","?","*",",","\t",""]
        for line in open(path):
            if line[-1] == "\n":
                line = line[:-1]
            a = int(line[-1])
            b=[]
            for word in line[:-1].split(' '):
                ##while word and word[-1] in punctuation:
                    ##word = word[:-1]
                ##b.append(wordnet_lemmatizer.lemmatize(word.lower()))
                i = 0
                while i < len(word):
                    if word[i] in punctuation:
                        word = word[:i]+word[i+1:]
                    else:
                        i+=1
                c = word.lower()
                if c == "and" or c == "or" or c=="":
                    continue
                b.append(c)
            if count[a] > 400:
                test_label.append(a)
                test_data.append(b)
            else:
                train_label.append(a)
                train_data.append(b)
            count[a]+=1
    return [train_data, train_label, test_data, test_label]

In [91]:
[train_data, train_label, test_data, test_label] = Split(["yelp_labelled.txt","amazon_cells_labelled.txt","imdb_labelled.txt"])

## Bag of Words 

In [92]:
"""
dic is a dictionary where key is the word shows in train_data and the items
of is a list with two elements, first one is the frequency of the key and 
second element is the index of the key in feature vector, which we will use
after.
"""
def bagOfWord(data):
    dic = {}
    t = 0
    n = 0
    for dataset in data:
        for line in dataset:
            for word in line:
                if word in dic:
                    dic[word][0] += 1
                elif t == 0:
                    dic[word] = [1,n]
                    n+=1
        t = 1
    return dic

In [93]:
Dic = bagOfWord([train_data, test_data])
len(Dic)

4819

In [94]:
"""Build feature vector."""
def buildB(data, dic):
    data_b = []
    size_dic = len(dic)
    for line in data:
        temp = [0]*size_dic
        for word in line:
            if word in dic:
                temp[dic[word][1]]+=1.0
        data_b.append(np.array(temp))
    return data_b
    

In [95]:
%time [train_data_b, test_data_b] = [buildB(train_data,Dic), buildB(test_data,Dic)]

CPU times: user 1.56 s, sys: 28.3 ms, total: 1.59 s
Wall time: 1.59 s


### postprocess feature vectors

In [96]:
"""
l^2 normalization
"""
def l2normalize(data):
    for vector in data:
        L = np.linalg.norm(vector)
        vector /= L
                
def standardize(data_b, size_dic):
    s = np.array([0.0]*size_dic)
    for bite in data_b:
        s += bite
    s_ = s/len(data_b)
    vec = []
    for bit in data_b:
            vec.append(bit - s_)
    return np.array(vec)

In [97]:
"""
train_vec and test_vec will be the feature vector to be used for future.
"""
l2normalize(train_data_b), l2normalize(test_data_b)
[train_vec, test_vec] = [standardize(train_data_b,len(Dic)), standardize(test_data_b,len(Dic))]

# K-means

In [98]:
"""
randomly pick two points in sample set to be initial points
label is a list indicate which cluster the vector is signed to.
p is the list including two mean point that the model converget to.
During the function, it first prints which two points function pick as
initial points and then how many time it iterates.
"""
def KMeans_2(data,size_dic):
    ##p = kmeans.cluster_centers_
    ##label = kmeans.labels_
    a = random.randint(0,len(data)-1)
    b = random.randint(0,len(data)-1)
    while a==b:
        b = random.randint(0,len(data)-1)
    p = np.array([data[a], data[b]])
    print("point_init1 is ",a)
    print("point_init2 is ",b)
    label = [0]*len(data)
    conver = False
    count = 0
    while not conver:
        count += 1
        conver = True
        for i in range(len(data)):
            d = [0]*2
            d[0] = np.linalg.norm(p[0]-data[i])
            d[1] = np.linalg.norm(p[1]-data[i])
            if d[label[i]] > d[1-label[i]]:
                conver = False
                label[i] = 1-label[i]
        if not conver:
            ##print("a")
            for j in [0,1]:
                n_p = 0
                s_p = np.array([0.0]*size_dic)
                for point in range(len(label)):
                    if label[point] == j:
                        s_p += data[point]
                        n_p += 1
                p[j] = s_p/n_p
    print("iterate time is ",count)
    return(label, p)

In [99]:
def n_kmeans(vec, k_train_label,kmeans_lib,size):
    [k_label, k_p]=KMeans_2(vec, size)
    n_bruce = 0
    n_python = 0
    for i in range(len(k_label)):
        if k_train_label[i] == kmeans_lib.labels_[i]:
            n_python+=1
        if k_train_label[i] == k_label[i]:
            n_bruce+=1
    print("self-designed accuracy is",n_bruce/len(k_label))
    print("          lib accuracy is", n_python/len(k_label))
    print("higher than lib?: ",n_python/len(k_label)<n_bruce/len(k_label) )
    print("************************")

In [100]:
kmeans = KMeans(n_clusters=2, random_state=0).fit(train_vec)

In [101]:

for i in range(3):
    n_kmeans(train_vec, train_label, kmeans,len(Dic))

point_init1 is  587
point_init2 is  1305
iterate time is  10
self-designed accuracy is 0.5054166666666666
          lib accuracy is 0.5058333333333334
higher than lib?:  False
************************
point_init1 is  694
point_init2 is  947
iterate time is  12
self-designed accuracy is 0.49541666666666667
          lib accuracy is 0.5058333333333334
higher than lib?:  False
************************
point_init1 is  1302
point_init2 is  844
iterate time is  13
self-designed accuracy is 0.51375
          lib accuracy is 0.5058333333333334
higher than lib?:  True
************************


### Logistic Regression

In [102]:
lgr = LGR()
lgr.fit(train_vec ,train_label)
lgr.score(test_vec,test_label)

0.79166666666666663

# N-gram model

In [103]:
def Ngram(data):
    data_ng = []
    for line in data:
        line_new = []
        for i in range(len(line)-1):
            line_new.append(line[i]+' '+line[i+1])
        data_ng.append(line_new)
    return data_ng

In [104]:
train_data_ng = Ngram(train_data)
test_data_ng = Ngram(test_data)

In [105]:
Dic_ng = bagOfWord([train_data_ng, test_data_ng])
len(Dic_ng)

16621

In [106]:
%time [train_data_ng_b, test_data_ng_b] = [buildB(train_data_ng,Dic_ng), buildB(test_data_ng,Dic_ng)]

CPU times: user 5.38 s, sys: 911 ms, total: 6.29 s
Wall time: 6.35 s


In [107]:
##drop empty element
train_label_ng = train_label[:]
i = 0
while i < len(train_data_ng_b):
    if not np.linalg.norm(train_data_ng_b[i]):
        train_label_ng.pop(i)
        train_data_ng_b.pop(i)
        train_data_ng.pop(i)
    else:
        i+=1
        
test_label_ng = test_label[:]
i = 0
while i < len(test_data_ng_b):
    if not np.linalg.norm(test_data_ng_b[i]):
        test_label_ng.pop(i)
        test_data_ng_b.pop(i)
        test_data_ng.pop(i)
    else:
        i+=1

In [108]:
l2normalize(train_data_ng_b)
l2normalize(test_data_ng_b)
[train_vec_ng, test_vec_ng] = [standardize(train_data_ng_b, len(Dic_ng)), standardize(test_data_ng_b, len(Dic_ng))]

In [109]:
kmeans_ng = KMeans(n_clusters=2, random_state=0).fit(train_vec_ng)

In [110]:
for i in range(3):
    n_kmeans(train_vec_ng, train_label_ng, kmeans_ng, len(Dic_ng))

point_init1 is  812
point_init2 is  2358
iterate time is  7
self-designed accuracy is 0.5121338912133891
          lib accuracy is 0.49874476987447697
higher than lib?:  True
************************
point_init1 is  1910
point_init2 is  2304
iterate time is  7
self-designed accuracy is 0.501673640167364
          lib accuracy is 0.49874476987447697
higher than lib?:  True
************************
point_init1 is  1830
point_init2 is  224
iterate time is  9
self-designed accuracy is 0.5359832635983264
          lib accuracy is 0.49874476987447697
higher than lib?:  True
************************


In [111]:
lgr = LGR()
lgr.fit(train_vec_ng ,train_label_ng)
lgr.score(test_vec_ng,test_label_ng)

0.7436823104693141

In [112]:
np.linalg.norm(sum(train_vec))

3.0703402281144394e-12

In [113]:
train_vec = np.array(train_vec)

## PCA

In [136]:
U, s, V = np.linalg.svd(train_vec, full_matrices=True)

In [137]:
s.shape, V.shape

((2400,), (4819, 4819))

In [118]:
train_vec[0].dot(V[:5].T)

array([-0.22109504,  0.07218909, -0.1547806 ,  0.12780164, -0.1292377 ])

In [119]:
def reduce(n):
    
    train_vec_n = train_vec.dot(V[:n].T)
    test_vec_n = test_vec.dot(V[:n].T)
    kmeans_n = KMeans(n_clusters=2, random_state=0).fit(train_vec_n)
    for i in range(3):
        n_kmeans(train_vec_n, train_label, kmeans_n, n)
    lgr = LGR()
    lgr.fit(train_vec_n ,train_label)
    print("Logistic Regression result is",lgr.score(test_vec_n,test_label))

In [120]:
reduce(10)

point_init1 is  2387
point_init2 is  1406
iterate time is  8
self-designed accuracy is 0.4945833333333333
          lib accuracy is 0.4945833333333333
higher than lib?:  False
************************
point_init1 is  1207
point_init2 is  1720
iterate time is  11
self-designed accuracy is 0.50625
          lib accuracy is 0.4945833333333333
higher than lib?:  True
************************
point_init1 is  1105
point_init2 is  656
iterate time is  10
self-designed accuracy is 0.5154166666666666
          lib accuracy is 0.4945833333333333
higher than lib?:  True
************************
Logistic Regression result is 0.531666666667


In [121]:
reduce(50)

point_init1 is  706
point_init2 is  1996
iterate time is  10
self-designed accuracy is 0.50625
          lib accuracy is 0.5045833333333334
higher than lib?:  True
************************
point_init1 is  14
point_init2 is  308
iterate time is  15
self-designed accuracy is 0.53
          lib accuracy is 0.5045833333333334
higher than lib?:  True
************************
point_init1 is  499
point_init2 is  84
iterate time is  12
self-designed accuracy is 0.5045833333333334
          lib accuracy is 0.5045833333333334
higher than lib?:  False
************************
Logistic Regression result is 0.678333333333


In [122]:
reduce(100)

point_init1 is  1220
point_init2 is  1866
iterate time is  10
self-designed accuracy is 0.46958333333333335
          lib accuracy is 0.505
higher than lib?:  False
************************
point_init1 is  576
point_init2 is  451
iterate time is  10
self-designed accuracy is 0.48583333333333334
          lib accuracy is 0.505
higher than lib?:  False
************************
point_init1 is  2270
point_init2 is  1235
iterate time is  9
self-designed accuracy is 0.48541666666666666
          lib accuracy is 0.505
higher than lib?:  False
************************
Logistic Regression result is 0.71


In [141]:
##compare with PCA Lib
from sklearn.decomposition import PCA
pca = PCA(n_components=100)
pca.fit(train_vec)
abs(abs(V[:100])-abs(pca.components_)) < 0.0000000000001

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ..., 
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]], dtype=bool)